In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
%matplotlib inline

In [ ]:
df = pd.read_csv('incomes.csv')
incomes = df['income'].values
degree  = df['has_degree'].values

## Problem 1: Finding the median income

In [ ]:
def get_random_indicies_resample(num):
    return np.random.randint(0, num, num)

def get_sample(X):
    indicies = get_random_indicies_resample(len(X))
    return X[indicies]

def get_sample_corr(X,y):
    indicies = get_random_indicies_resample(len(X))
    return X[indicies], y[indicies]

def get_sample_median(X):
    sample = get_sample(X)
    return np.median(sample)

In [ ]:
medians = [get_sample_median(incomes) for _ in range(200)]

In [ ]:
plt.figure(dpi=200)
plt.hist(medians);
plt.title("Distribution of medians")
plt.xlabel("Median income of sample")
plt.ylabel("Number of samples")

whole_sample_median = np.median(incomes)
plt.gca().axvline(whole_sample_median, ls='--', c='k');

In [ ]:
# What are the percentiles? This gives us a range of values 
# that the sample median lies in 95% of the samples
np.percentile(medians,2.5), np.percentile(medians, 97.5)

In [ ]:
plt.figure(dpi=200)
plt.hist(medians);
plt.title("Distribution of medians")
plt.xlabel("Median income of sample")
plt.ylabel("Number of samples")

whole_sample_median = np.median(incomes)
lower, higher = np.percentile(medians,2.5), np.percentile(medians, 97.5)

axis = plt.gca()
axis.axvline(whole_sample_median, ls='--', c='k');
axis.axvline(lower, ls='--', c='b');
axis.axvline(higher, ls='--', c='b');

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
coefficients, intercepts = [], []

for sample in range(200):
    lm = LogisticRegression()
    sampleX, sampleY = get_sample_corr(incomes, degree)
    lm.fit(sampleX.reshape((-1,1)), sampleY)
    coefficients.append(lm.coef_[0,0])
    intercepts.append(lm.intercept_[0])

In [ ]:
import matplotlib.ticker as ticker

plt.figure(dpi=100)
plt.plot(coefficients, intercepts,'o')
plt.plot()
plt.xlabel('coefficient')
plt.ylabel('intercept')
plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.0e'));

Standard scaling is really important! Look at what the difference is when we include it!

In [ ]:
from sklearn.preprocessing import StandardScaler

coefficients, intercepts = [], []

for sample in range(200):
    lm = LogisticRegression()
    ssX = StandardScaler()
    sampleX, sampleY = get_sample_corr(incomes, degree)
    sampleX_scaled = ssX.fit_transform(sampleX.reshape((-1,1)), sampleY)
    lm.fit(sampleX_scaled, sampleY)
    
    # This converts from the coefficients on the scaled version to
    # coefficients in the "original" coordinates
    raw_coef = lm.coef_[0,0]/np.sqrt(ssX.var_[0])
    raw_intercept = lm.intercept_[0] - raw_coef*ssX.mean_[0]
    coefficients.append(raw_coef)
    intercepts.append(raw_intercept)

In [ ]:
plt.figure(dpi=100)
plt.plot(coefficients, intercepts,'o')

# This is a actual beta and b I used to generate the data
plt.plot([1/30000.0], [-4/3.0], 'bx')

plt.xlabel('coefficient')
plt.ylabel('intercept')
plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.0e'));